In [1]:
from azure.storage.table import TableService, Entity
from azure.servicebus import ServiceBusService, Message, Queue, Rule, DEFAULT_RULE_NAME
from azure.storage.queue import QueueService
import time

In [2]:
bus_service=ServiceBusService(
    service_namespace='Dream22',
    shared_access_key_name='RootManageSharedAccessKey',
    shared_access_key_value='eykA9ft6Ghg621smEDZK8uc0Y38EtgWBNZ6pCMzebvA=')

In [10]:
#bus_service.create_queue('vehiclecheck')

In [4]:
table_service = TableService(account_name='speedcameradata', account_key='BLPs0WUvgQkCBAAMF0+xi3wsROREeACdANAJEqlnD1xV2xN5/RqHDX4DwZpscQY2DnGJUg6x7eKmosHtt6SifQ==')
table_service.create_table('speedingvehicles')

False

In [5]:
bus_service.create_subscription('car_sightings', 'SpeedingCars')

rule = Rule()
rule.filter_type = 'SqlFilter'
rule.filter_expression = 'speed > speedlimit'

bus_service.create_rule('car_sightings', 'SpeedingCars', 'SpeedingCarsFilter', rule)
bus_service.delete_rule('car_sightings', 'SpeedingCars', DEFAULT_RULE_NAME)

True

In [9]:
# Set the interval between calls to the subscription while there are still messages (in seconds):

interval=0.5
delay=5

while True:

# Getting messages from subscription

    msg = bus_service.receive_subscription_message('car_sightings','SpeedingCars', peek_lock=False)

# Assigning variables
    
    try:
    
        camera= msg.custom_properties['cameraid']
        registration= msg.custom_properties['registration']
        vtype= msg.custom_properties['type']
        speed= msg.custom_properties['speed']
        town= msg.custom_properties['town']
        street= msg.custom_properties['street']
        speedlimit= msg.custom_properties['speedlimit']

    # Setting priority    

        priority='False'
        if speed*0.9>=speedlimit:
            print('PRIORITY')
            priority='True'

    # Printing to terminal

        print('Camera ID: '+camera)
        print('Registration: '+registration)
        print('Street: '+street)
        print('Town/City: '+town)
        print('The vehicle was travelling at '+str(speed)+' in a '+str(speedlimit)+' zone.')
        print('----')
        print()

    # Sending to table storage

        task = Entity()
        task.PartitionKey = 'Camera'+str(msg.custom_properties['cameraid'])
        task.RowKey = str(time.strftime("%H.%M.%S-%d.%m.%Y-"))+
        str(msg.custom_properties['registration'][0:4])+str(msg.custom_properties['registration'][5:])
        task.camera=msg.custom_properties['cameraid']
        task.registration = msg.custom_properties['registration']
        task.vehtype = msg.custom_properties['type']
        task.speed=msg.custom_properties['speed']
        task.town=msg.custom_properties['town']
        task.street=msg.custom_properties['street']
        task.speedlimit=msg.custom_properties['speedlimit']
        task.priority=priority

        table_service.insert_entity('speedingvehicles', task)
        
        msg=Message(bytes({0}),custom_properties={"cameraid":camera,"street":street,"town":town,"speedlimit":speedlimit,"registration":registration,"type":vtype,"speed":speed,"Priority":priority})
        bus_service.send_queue_message('vehiclecheck', msg)
    
    except:
        print('No more messages')
        print('----')
        time.sleep(delay)

    
# Sending to vehicle check queue
    
    
    

    time.sleep(interval)
        


Camera ID: 46638
Registration: WG29 PCR
Street: Cathedral Road
Town/City: Cardiff
The vehicle was travelling at 41 in a 40 zone.
----

PRIORITY
Camera ID: 12345
Registration: OK67 BRJ
Street: Westgate
Town/City: Dundee
The vehicle was travelling at 71 in a 60 zone.
----

PRIORITY
Camera ID: 67891
Registration: SN60 YYU
Street: St James Blvd
Town/City: Newcastle
The vehicle was travelling at 56 in a 40 zone.
----

PRIORITY
Camera ID: 46638
Registration: KT38 HMK
Street: Cathedral Road
Town/City: Cardiff
The vehicle was travelling at 49 in a 40 zone.
----

PRIORITY
Camera ID: 47829
Registration: KL56 TSS
Street: Great Ancoats Street
Town/City: Manchester
The vehicle was travelling at 68 in a 50 zone.
----

PRIORITY
Camera ID: 92658
Registration: CE21 QDU
Street: Crescent Link
Town/City: Londonderry
The vehicle was travelling at 69 in a 50 zone.
----

PRIORITY
Camera ID: 47829
Registration: KR34 SEX
Street: Great Ancoats Street
Town/City: Manchester
The vehicle was travelling at 64 in a 5

KeyboardInterrupt: 